In [1]:
import gym
env = gym.make('CartPole-v0')
env.reset()
#random
for i_episode in range(1):
    observation = env.reset()
    for _ in range(100):
        env.render()
        step= env.action_space.sample()
        observation,reward,done,info= env.step(step) # take a random action

        #print(step,observation,reward)
#         if done:
#             print("episode finished after {} timesteps".format(_+1))
#             break
env.close()

C:\Users\csviv\Anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
